In [1]:
!pip install ncps

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 44.8/44.8 kB 833.7 kB/s eta 0:00:00


# EEG Emotion Detection - NCPS

---


****
**Importing Libraries**

Importing necessary libraries.
****

In [2]:
import numpy as np
import pandas as pd
import seaborn as sns
import tensorflow as tf
import matplotlib.pyplot as plt
import keras
from ncps.wirings import AutoNCP
from ncps.tf import LTC
import tensorflow as tf
from ncps import wirings
import tensorflow as tf
from sklearn.utils.class_weight import compute_class_weight
from keras.callbacks import EarlyStopping, ModelCheckpoint

****
**Uploading The Data**

We will be using the dataset "emotions.csv" by Jordan Bird
****

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
data = pd.read_csv('/content/drive/MyDrive/emotions.csv')
len(data)

****
**Viewing the time series data for the row 1, column fft_0_b to fft_749_b**

Since we are certain that the data from column fft_0_a to fft_749_b are time series data, we are going to make use of the data for our machine learning.
****

In [ ]:
sample = data.loc[0, 'fft_0_b':'fft_749_b']

plt.figure(figsize=(16, 10)) #figure size
plt.plot(range(len(sample)), sample) #range x = length of sample, y = sample
plt.title("Features fft_0_b to fft_749_b")
plt.show()

****
**Counting the total for each label in the column label**

The column named label consists of NEUTRAL, POSITIVE, and NEGATIVE
****

In [ ]:
sample = data.iloc[0]
label_mapping = {label: i for i, label in enumerate(data['label'].unique())}

In [ ]:
label_mapping
data['label'] = data['label'].map(label_mapping)

In [ ]:
data

In [ ]:
data['label'].value_counts()

****
**Checking for null values**

Checking for missing values
****

In [ ]:
data.isnull().sum()

****
**Labelling the label provided with an integer**

NEGATIVE will be labelled as 0, NEUTRAL as 1, POSITIVE as 2
****

In [ ]:
label_mapping = {'NEGATIVE': 0, 'NEUTRAL': 1, 'POSITIVE': 2}
data['label'] = data['label'].replace(label_mapping)

In [ ]:
data

****
**Defining X and y**

X will contain the original data but without the label column, while y will contain only the label column.
****

In [ ]:
X = data.drop('label', axis=1)
y = data['label'].copy()

****
**Preprocessing The Data**

Preprocessing X and y.
****

In [ ]:
from sklearn.preprocessing import StandardScaler
scaler = StandardScaler()
scaler.fit(X)
X = scaler.transform(X)

from tensorflow.keras.utils import to_categorical
y = to_categorical(y)

In [ ]:
print (y)
print (type(y))

****
**Splitting The Data of X and y**

Split arrays or matrices of X and y into random train and test subsets.
****

In [ ]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, train_size = 0.8, random_state = np.random)

In [ ]:
X_train.shape

****
**Reshaping X_train and X_test**

Reshaping X_train and X_test without changing its data.
****

In [ ]:
X_train = np.reshape(X_train, (X_train.shape[0],1,X.shape[1]))
X_test = np.reshape(X_test, (X_test.shape[0],1,X.shape[1]))

****
**NCPS Implementation**

Implementing NCPS classification to the data.
****

In [ ]:
X_train.shape

In [ ]:
X_train.shape[2]

In [ ]:
X_test.shape

In [ ]:
X_test[0]

In [ ]:
print(dir(y_train))

In [ ]:
# X_train = X_train.reshape(X_train.shape[0], 1, X_train.shape[1])
# X_test = X_test.reshape(X_test.shape[0], 1, X_test.shape[1])
import keras
from ncps.wirings import AutoNCP
from ncps.tf import LTC
import tensorflow as tf
from ncps import wirings
import tensorflow as tf
from sklearn.utils.class_weight import compute_class_weight
from keras.callbacks import EarlyStopping, ModelCheckpoint
# import kerasncp as kncp
# from kerasncp.tf import LTCCell
# from ncps import AutoNCP

num_classes = 3
timesteps = 1
input_shape = (timesteps, X_train.shape[2])


# arch = kncp.wirings.Random(32, 1, sparsity_level=0.5)  # 32 units, 1 motor neuron
# rnn_cell = LTCCell(arch)

#wiring = kncp.wirings.FullyConnected(10)
wiring = wirings.AutoNCP(100, 15)
rnn_cell = LTC(wiring)
# cell = MinimalRNNCell(32)

model = keras.models.Sequential(
    [
        keras.layers.InputLayer(input_shape=input_shape),
        keras.layers.Conv1D(filters=82, kernel_size=3, activation='relu', padding='causal'),
        # keras.layers.RNN(rnn_cell, return_sequences=True),
        # LTC(wiring, return_sequences=True),
        # keras.layers.RNN(rnn_cell, return_sequences=False, return_state=False, go_backwards=False, stateful=False, unroll=False),
        LTC(wiring, return_sequences=True),
        keras.layers.GlobalAveragePooling1D(),
        keras.layers.Dense(num_classes, activation="softmax"),
    ]
)


# model.compile(
#     optimizer=keras.optimizers.Adam(0.01),
#     loss='sparse_categorical_crossentropy',
#     metrics=['accuracy'],
#     #run_eagerly=True
# )

model.summary()

model.compile(loss = 'categorical_crossentropy', optimizer = "adam", metrics = ['accuracy'])

# callbacks
early_stopping = EarlyStopping(monitor='val_accuracy', patience=100, verbose=1, mode='max')
model_checkpoint = ModelCheckpoint('test.h5', monitor='val_accuracy', mode='max', verbose=1, save_weights_only=True)

y_train_encoded = to_categorical(y_train)
y_test_encoded = to_categorical(y_test)

# model training
history = model.fit(X_train, y_train, epochs=100, batch_size=30, validation_data=(X_test, y_test), shuffle=True, callbacks=[early_stopping, model_checkpoint])

In [ ]:
y_train.shape[1]

In [ ]:
yhat = model.predict(X_train[:1, :])
print(yhat)

In [ ]:
keras.__version__

In [ ]:
tf.__version__

In [ ]:
model.load_weights("/content/test.h5")

In [ ]:
import os
checkpoint_callback = ModelCheckpoint(os.path.join('models','checkpoint'), monitor='loss', save_weights_only=True)

# ****
**NCPS Summary**

Printing NCPS application summary.
****

****
**Fit Training The Data Using NCPS**

Testing the NCPS.
****

****
**Test Accuracy**

Calculating test accuracy.
****

In [ ]:
from sklearn.metrics import accuracy_score
pred = model.predict(X_test)
predict_classes = np.argmax(pred,axis=1)
expected_classes = np.argmax(y_test,axis=1)

model_acc = accuracy_score(expected_classes,predict_classes)
print("Test Accuracy: {:.3f}%".format(model_acc * 100))

In [ ]:
X_test


In [ ]:
def predictn(row):
  pred1 = model.predict(X_test[row:row+1, :])
  prediction = np.argmax(pred1,axis=1)
  print(pred1)
  print(prediction)

****
**Confusion Matrix**

Plotting confusion matrix.
****

In [ ]:
from sklearn.metrics import confusion_matrix
prediction = model.predict(X_test)
cm = confusion_matrix(prediction.argmax(axis=1),y_test.argmax(axis=1))

print("Confusion Matrix:\n",(cm))

plt.figure(figsize=(8, 8))
sns.heatmap(cm, annot=True, vmin=0, fmt='g', cbar=False, cmap='Blues')
plt.xticks(np.arange(3) + 0.5, label_mapping.keys())
plt.yticks(np.arange(3) + 0.5, label_mapping.keys())
plt.xlabel("Predicted")
plt.ylabel("Actual")
plt.title("Confusion Matrix")
plt.show()

****
**Classification Report**

Viewing classification report.
****

In [ ]:
from sklearn.metrics import classification_report
clr = classification_report(prediction.argmax(axis=1), y_test.argmax(axis=1), target_names=label_mapping.keys())

print("Classification Report:\n----------------------\n", clr)

Classification Report:
----------------------
               precision    recall  f1-score   support

    NEGATIVE       1.00      0.89      0.94       154
     NEUTRAL       1.00      0.97      0.99       147
    POSITIVE       0.86      1.00      0.92       126

    accuracy                           0.95       427
   macro avg       0.95      0.95      0.95       427
weighted avg       0.96      0.95      0.95       427



In [ ]:
model.save("NCPS.h5")

In [ ]:
import numpy as np

arr = np.array([[1, 2, 3],
                [4, 5, 6]])

# Find the index of the maximum value in the flattened array
index_flattened = np.argmax(arr)
print("Index of maximum value (flattened):", index_flattened)  # Output: 5

# Find the index of the maximum value along axis 0 (columns)
index_axis_0 = np.argmax(arr, axis=0)
print("Index of maximum value along axis 0:", index_axis_0)  # Output: [1 1 1]

# Find the index of the maximum value along axis 1 (rows)
index_axis_1 = np.argmax(arr, axis=1)
print("Index of maximum value along axis 1:", index_axis_1)  # Output: [2 2]

Index of maximum value (flattened): 5
Index of maximum value along axis 0: [1 1 1]
Index of maximum value along axis 1: [2 2]
